<a href="https://colab.research.google.com/github/jlopetegui98/Creation-of-a-synthetic-dataset-for-French-NER-in-clinical-trial-texts/blob/main/TranslationModel/translation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 5.9 MB/s eta 0:00:00


In [3]:
# uncomment the following line to run in colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import utils
import os
from tqdm import tqdm

In [5]:
# DATA
# data_path = '../data' # comment if using colab
data_path = './drive/MyDrive/HandsOn-NLP/data' # uncomment if using colab
chia_path = f'{data_path}/chia_bio'
chia_texts_path = f'{data_path}/chia_criteria_texts'
chia_french_trans_path = f'{data_path}/chia_criteria_french'

In [6]:
# read chia dataset
files = os.listdir(chia_texts_path)
files = [f for f in files if f.endswith('.txt')]
len([file for file in files if file.endswith('exc.bio.txt')]), len([file for file in files if file.endswith('inc.bio.txt')])

(1000, 1000)

In [11]:
# generate for each file the corresponding file with the sentences for each criteria
for file in tqdm(files):
    with open(f'{chia_path}/{file}', 'r') as f:
        file_text = f.read()
    sentences = utils.get_criteria_str_list(file_text)
    with open(f'{data_path}/chia_criteria_texts/{file}', 'w') as f:
        f.write('\n'.join(sentences))

### Translation of each sentence using hugginface model for translation

In [7]:
# get list of already translated files
translated_trials = os.listdir(chia_french_trans_path)
len(translated_trials)

1028

In [8]:
translated_trials[0], files[0] # to see the difference in file names

('NCT02924090_exc.bio_fr.txt', 'NCT03134378_inc.bio.txt')

In [10]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr", device = 0)

# translate the sentences
for file in tqdm(files):
    if file.split('.txt')[0]+'_fr.txt' in translated_trials:
      continue
    with open(f'{data_path}/chia_criteria_texts/{file}', 'r') as f:
        file_text = f.read()
    print(file)
    sentences = file_text.split('\n')
    sentences = [s for s in sentences if s if len(s) > 0 and len(s.split()) < 512]
    translated_sentences = translator(sentences)
    with open(f'{data_path}/chia_criteria_french/{file.replace(".txt", "_fr.txt")}', 'w') as f:
        f.write('\n'.join([t['translation_text'] for t in translated_sentences]))